# Conversational UI Chatbot App with ChatGPT, LangChain and Chainlit

Here we will build a advanced ChatGPT Conversational UI-based chatbot using LangChain and Chainlit with the following features:

- Custom Landing Page
- Conversational memory
- Result streaming capabilities (Real-time output)

## Install App and LLM dependencies

In [ ]:
!pip install langchain==0.1.12
!pip install langchain-openai==0.0.8
!pip install chainlit==1.0.401
!pip install pyngrok==7.1.5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.1/809.1 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.1/269.1 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 15.8 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.9/262.9 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

## Load OpenAI API Credentials

Here we load it from a file so we don't explore the credentials on the internet by mistake

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
import yaml

with open('chatgpt_api_credentials.yml', 'r') as file:
    api_creds = yaml.safe_load(file)

In [ ]:
api_creds.keys()

dict_keys(['openai_key'])

In [ ]:
import os

os.environ['OPENAI_API_KEY'] = api_creds['openai_key']

## Write the app code here and store it in a py file

In [ ]:
# the following line is a magic command
# that will write all the code below it to the python file app.py
# we will then deploy this app.py file on the cloud server where colab is running
# if you have your own server you can just write the code in app.py and deploy it directly
%%writefile app.py

from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferWindowMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain.schema.runnable.config import RunnableConfig
from langchain.schema import StrOutputParser
from operator import itemgetter
import chainlit as cl

@cl.on_chat_start
# this function is called when the app starts for the first time
async def when_chat_starts():

  # Load a connection to ChatGPT LLM
  chatgpt = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.1,
                       streaming=True)

  # Add a basic system prompt for LLM behavior
  SYS_PROMPT = """
               Act as a helpful assistant and answer questions to the best of your ability.
               Do not make up answers.
               """

  # Create a prompt template for langchain to use history to answer user prompts
  prompt = ChatPromptTemplate.from_messages(
    [
      ("system", SYS_PROMPT),
      MessagesPlaceholder(variable_name="history"),
      ("human", "{input}"),
    ]
  )

  # Create a memory object to store conversation history window
  memory = ConversationBufferWindowMemory(k=20,
                                          return_messages=True)

  # Create a conversation chain
  conversation_chain = (
    RunnablePassthrough.assign(
      history=RunnableLambda(memory.load_memory_variables)
      |
      itemgetter("history")
    )
    |
    prompt
    |
    chatgpt
    |
    StrOutputParser() # to parse the output to show on UI
  )
  # Set session variables to be accessed when user enters prompts in the app
  cl.user_session.set("chain", conversation_chain)
  cl.user_session.set("memory", memory)


@cl.on_message
# this function is called whenever the user sends a prompt message in the app
async def on_user_message(message: cl.Message):

  # get the chain and memory objects from the session variables
  chain = cl.user_session.get("chain")
  memory = cl.user_session.get("memory")

  # this will store the response from ChatGPT LLM
  chatgpt_message = cl.Message(content="")

  # Stream the response from ChatGPT and show in real-time
  async for chunk in chain.astream(
    {"input": message.content},
    config=RunnableConfig(callbacks=[cl.LangchainCallbackHandler()]),
  ):
      await chatgpt_message.stream_token(chunk)
  # Finish displaying the full response from ChatGPT
  await chatgpt_message.send()
  # Store the current conversation in the memory object
  memory.save_context({"input": message.content},
                      {"output": chatgpt_message.content})


Writing app.py


## Start the app

In [ ]:
!chainlit run app.py --port=8989 --watch &>./logs.txt &

In [ ]:
from pyngrok import ngrok
import yaml

# Terminate open tunnels if exist
ngrok.kill()

# Setting the authtoken
# Get your authtoken from `ngrok_credentials.yml` file
with open('ngrok_credentials.yml', 'r') as file:
    NGROK_AUTH_TOKEN = yaml.safe_load(file)
ngrok.set_auth_token(NGROK_AUTH_TOKEN['ngrok_key'])

# Open an HTTPs tunnel on port XXXX which you get from your `logs.txt` file
ngrok_tunnel = ngrok.connect(8989)
print("Chainlit App:", ngrok_tunnel.public_url)

Chainlit App: https://7df0-35-227-140-75.ngrok-free.app


## Change the Initial app screen

In [ ]:
%%writefile chainlit.md

# Welcome I am AI Assistant 🤖

How can I help you today?

Overwriting chainlit.md


## Remove running app processes

In [ ]:
ngrok.kill()

In [ ]:
!ps -ef | grep app

root           7       1  0 23:28 ?        00:00:07 /tools/node/bin/node /datalab/web/app.js
root        5617       1  2 23:51 ?        00:00:10 /usr/bin/python3 /usr/local/bin/chainlit run app
root        7121     498  0 23:57 ?        00:00:00 /bin/bash -c ps -ef | grep app
root        7123    7121  0 23:57 ?        00:00:00 grep app


In [ ]:
!sudo kill -9 5617